# Manipuler les données avec Pandas pour aller vers l'analyse statistique

En un mot : Pandas est une gamme d'outils pour manipuler des tableaux, et aller jusqu'à l'analyse des données

Cela simplifie beaucoup de traitements, surtout pour l'exploration.

## Avant Pandas : avoir un tableau de données

Articles du Monde, quotidien français, sur une période, mentionnant le terme "islamogauchisme" : construire un unique fichier CSV qui comprend : la date de l'article, son titre, le nombre de mots qu'il contient, et le nombre de fois où le terme "islamogauchisme" ou "islamo-gauchisme" est contenu.

On peut le faire uniquement avec les outils déjà vu. Nous allons utiliser Pandas pour faire ça dans ce cours, puis analyser les données

In [1]:
import os
import pandas as pd

# Fonction pour compter
def compter(x):
    mots = ["islamogauchisme","islamo-gauchisme"]
    total = 0
    for m in mots:
        total+=x.lower().count(m)
    return total

# Créer le corpus
corpus = []
for fichier in os.listdir("./Data/articles/"):
    date = fichier[0:10]
    titre = fichier[12:]
    with open("./Data/articles/"+fichier,"r") as f:
        contenu = f.read()
    total_titre_ig = compter(titre)
    total_contenu_ig = compter(contenu)
    total_mots = len(contenu.split())
    corpus.append([date,titre,contenu,total_mots,total_titre_ig,total_contenu_ig])

corpus = pd.DataFrame(corpus, columns=['date','titre','contenu','total_mots','total_titre_ig','total_contenu_ig'])
corpus.to_excel("Data/tableau.xls", index=False)

/var/folders/ld/jy0c_5d91rl_s3jqh74tmqn00000gn/T/ipykernel_1848/1262774784.py:25: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  corpus.to_excel("Data/tableau.xls", index=False)


## Manipuler un tableau Pandas

In [2]:
import pandas as pd
data = pd.read_excel("./Data/extraction-ig-europresse.xls")

### Sélectionner les éléments : colonnes, lignes, cases

### Mettre en forme : colonne et index

### Filtrer le tableau

### Rajouter des colonnes

Recoder les journaux ; Rajouter une colonne nombre de mots ; une colonne présence des termes ; une colonne densité du terme

In [4]:
import regex as re
import datetime

def reco_date(x):
    #conversion lettres/chiffres
    mois = {"janvier":"01","février":"02","mars":"03",
            "avril":"04","mai":"05","juin":"06","juillet":"07",
            "août":"08","septembre":"09","octobre":"10","novembre":"11","décembre":"12"}
    
    #Trouver le motif
    t = re.findall("\w+ \w+ [0-9]{4}",x)
    
    #Si pas trouvé, renvoyer rien
    if len(t) <1:
        return None
    
    #Sinon prendre le premier élémen
    t
    t = t[0]
    
    #Remplacer le mois pour une mise en forme
    for i in mois:
        if i in t:
            t = t.replace(i,"/%s/"%mois[i]).replace(" ","")

    #Mettre sous la forme d'un objet datetime 
    t = datetime.datetime.strptime(t,"%d/%m/%Y")
            
    return t

In [6]:
data["nombre_ig"] = data["Contenu"].apply(compter)
data["nombre_mots"] = data["Contenu"].apply(lambda x: len(x.split()))
data["densite"] = data.apply(lambda x : None if x["nombre_mots"]==0 else x["nombre_ig"]/x["nombre_mots"],axis=1)
data["Date-reco"] = data["Date"].apply(reco_date)
def reco_journal(x):
    journals = {"Le Figaro":"Figaro","Humanité":"L'Humanité","Le Monde":"Le Monde",
                "La Croix":"La Croix","La Tribune":"La Tribune","Les Echos":"Les Echos",
               "Aujourd'hui en France":"Aujourd'hui en France"}
    for j in journals:
        if j in x:
            return journals[j]
    return x

data["Journal"] = data["Journal"].apply(reco_journal)

data.to_excel("Data/tableau_statistiques.xlsx",index=False)

### Créer un tableau Pandas de 0

### Sauvegarder un tableau Pandas

## Début de mise en forme

### Le tri à plat

### Les tableaux croisés

### Ensuite ? Mettre en forme ...

Exemple d'une fonction plus complète

In [1]:
def tri_a_plat(colonne):
    effectif = colonne.value_counts()
    pourcentage = round(100*colonne.value_counts(normalize=True),1)
    tableau = pd.DataFrame([effectif,pourcentage]).T
    tableau.columns = ["Effectif","Pourcentage (%)"]
    return tableau

def tableau_croise(c1,c2):
    t_absolu = pd.crosstab(c1,c2,margins=True)
    t_pourcentage = pd.crosstab(c1,c2)\
            .apply(lambda x: 100*x/sum(x),axis=1)
    t = t_absolu.copy()
    for i in range(0,t_pourcentage.shape[0]):
        for j in range(0,t_pourcentage.shape[1]):
            t.iloc[i,j] = str(t_absolu.iloc[i,j]) \
            +" ("+str(round(t_pourcentage.iloc[i,j],1))+"%)"
    return t

## Exemple d'utilisation avancée

- multiple index
- join
- merge
- concat
- etc.

In [36]:
bdd = pd.DataFrame([["Le Monde","sdsdsd"],["Le Monde","qsdsddsq"],["Libération","qsd0"]],columns = ["Journal","Texte"])
info = pd.DataFrame([["Le Monde","10"],["Libération","20"]],columns = ["Journal","Nombre"]).set_index("Journal")
bdd.join(info,on="Journal")

,Journal,Texte,Nombre
0,Le Monde,sdsdsd,10
1,Le Monde,qsdsddsq,10
2,Libération,qsd0,20
